# PODCAST SUMMARIZER USING PYTHON  SCRIPT

Importing libraries


In [2]:
#importing libraries
import pandas as pd
import transformers
import openai
from transformers import BartTokenizer, BartForConditionalGeneration
import os

Parsing our data and reading the first five rows

In [11]:
#specifying file path
text_data = pd.read_csv("116_diarized_timestamped_transcript.txt", delimiter="\t")
text_data

,"Speaker 0 (00:00:00,039-00:00:06,457): Podcast. Take it out. The Joe, Regan. Experience. Train my day, Joe Regan podcast, my day."
0,"Speaker 1 (00:00:11,273-00:00:13,492): George,..."
1,"Speaker 0 (00:00:14,959-00:00:15,399): Thank you."
2,"Speaker 1 (00:00:15,399-00:00:16,579): What do..."
3,"Speaker 0 (00:00:16,878-00:00:21,456): Yeah. L..."
4,"Speaker 1 (00:00:21,916-00:00:24,455): Does it..."
...,...
1013,"Speaker 0 (01:57:11,752-01:57:26,583): I reall..."
1014,"Speaker 1 (01:57:26,583-01:57:31,260): Well, G..."
1015,"Speaker 0 (01:57:31,260-01:57:31,580): you, sir."
1016,"Speaker 1 (01:57:31,580-01:57:34,619): Susan. ..."


In [12]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

# Download necessary NLTK resources if not already present
nltk.download('punkt')

# Define the transcript text
transcript = "116_diarized_timestamped_transcript.txt"

# Tokenize the transcript into sentences
sentences = sent_tokenize(transcript)

# Create a frequency distribution of words, excluding stopwords
stopwords = nltk.corpus.stopwords.words('english')
word_counts = FreqDist(word.lower() for sentence in sentences for word in sentence.split() if word not in stopwords)

# Identify the most frequent words (excluding punctuation)
most_frequent_words = word_counts.most_common(20)
non_punctuation_words = [word for word, _ in most_frequent_words if not any(char.isalnum() for char in word)]

# Extract candidate sentences using keyword matching
candidate_sentences = []
for sentence in sentences:
    # Check if at least 2 of the most frequent words are present
    if sum(word in sentence.lower() for word in non_punctuation_words) >= 2:
        candidate_sentences.append(sentence)

# Rank candidate sentences based on their position in the transcript (earlier sentences are considered more relevant)
ranked_sentences = sorted(candidate_sentences, key=sentences.index)

# Extract top N sentences (adjust N as needed)
summary = " ".join(ranked_sentences[:5])

# Print the summary
print(summary)


[nltk_data] Downloading package punkt to /home/scientist-
[nltk_data]     ezzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
